## FINDING THE BEST LOCATION FOR A TUITION CENTRE IN MELBOURNE, AUSTRALIA

### INSTALL AND IMPORT PACKAGES

In [1]:
# Install required packages
!pip install geopy
!pip install folium

     |████████████████████████████████| 94 kB 3.3 MB/s eta 0:00:011


In [2]:
# Import required packages
import pandas as pd
import numpy as np
import types
from botocore.client import Config
import ibm_boto3
import io
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
#from sklearn.cluster import KMeans
import folium
import requests
from project_lib import Project

### POSTCODE DATA WRANGLING

In [3]:
# The code was removed by Watson Studio for sharing.

,postcode,suburb,state,dc,type,lat,lon
0,200,AUSTRALIAN NATIONAL UNIVERSITY,ACT,AUSTRALIAN NATIONAL UNI LPO,Post Office Boxes,-35.277272,149.117136
1,221,BARTON,ACT,NaN,LVR,-35.201372,149.095065
2,800,DARWIN,NT,DARWIN DELIVERY CENTRE,Delivery Area,-12.801028,130.955789
3,801,DARWIN,NT,DARWIN DELIVERY CENTRE,Post Office Boxes,-12.801028,130.955789
4,804,PARAP,NT,PARAP,Post Office Boxes,-12.432181,130.843310


In [4]:
#Drop irrelevant columns
df_suburbdata = df_data_1
df_suburbdata = df_suburbdata.drop(['state', 'dc','type'], axis = 1)
df_suburbdata

#Keep only the rows corresponding to Metropolitan Melbourne
df_melbsuburbs = df_suburbdata.loc[((df_suburbdata['postcode'] >= 3000) & (df_suburbdata['postcode'] <= 3207))|((df_suburbdata['postcode'] >= 8000) & (df_suburbdata['postcode'] <= 8873))]
df_melbsuburbs = df_melbsuburbs.reset_index(drop=True)

#Preview the database to ensure it has been transformed as necessary
df_melbsuburbs.head()

,postcode,suburb,lat,lon
0,3000,MELBOURNE,-37.814563,144.970267
1,3001,MELBOURNE,-38.365017,144.765920
2,3002,EAST MELBOURNE,-37.816640,144.987811
3,3003,WEST MELBOURNE,-37.806255,144.941123
4,3004,MELBOURNE,-37.837324,144.976335


In [5]:
#Check to see if any nulls exist in the dataset.
df_melbsuburbs.isnull().sum()

postcode    0
suburb      0
lat         0
lon         0
dtype: int64

In [6]:
#Check for duplicate postcodes
df_melbsuburbs[df_melbsuburbs['postcode'].duplicated(keep=False)].head()

,postcode,suburb,lat,lon
9,3011,FOOTSCRAY,-37.799770,144.899587
10,3011,SEDDON,-37.808769,144.895486
11,3011,SEDDON WEST,-37.795059,144.866197
12,3012,BROOKLYN,-37.814624,144.847108
13,3012,KINGSVILLE,-37.812635,144.881803


In [7]:
#Group each suburb by unique postcode, creating a concatenated list of suburbs belonging to each postcode
df_aggsuburbs = df_melbsuburbs.groupby('postcode')['suburb'].apply(', '.join).to_frame()
df_aggsuburbs = df_aggsuburbs.reset_index()

#Average the latitude and longitude locations for each postcode
df_aveloc = df_melbsuburbs.groupby('postcode').mean()
df_aveloc = df_aveloc.reset_index()
df_melbpostcodes = pd.merge(df_aggsuburbs,df_aveloc)

#Preview the new table
df_melbpostcodes.head()

,postcode,suburb,lat,lon
0,3000,MELBOURNE,-37.814563,144.970267
1,3001,MELBOURNE,-38.365017,144.765920
2,3002,EAST MELBOURNE,-37.816640,144.987811
3,3003,WEST MELBOURNE,-37.806255,144.941123
4,3004,MELBOURNE,-37.837324,144.976335


### SCHOOL ENROLMENT DATA WRANGLING

In [8]:
# The code was removed by Watson Studio for sharing.

,Row Labels,Prep,Year 1,Year 2,Year 3,Year 4,Year 5,Year 6,Primary Ungraded,Year 7,Year 8,Year 9,Year 10,Year 11,Year 12,Secondary Ungraded,Special,Language,Grand Total
0,Abbotsford Primary School,24.0,27.0,28.0,27.0,18.0,8.0,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,144.0
1,Aberfeldie Primary School,48.0,61.0,52.0,62.0,67.0,59.0,49.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,398.0
2,Ainslie Parklands Primary School,14.0,19.0,20.0,14.0,26.0,24.0,30.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,147.0
3,Aireys Inlet Primary School,8.0,8.0,12.0,13.0,16.0,19.0,14.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,90.0
4,Airly Primary School,2.0,5.0,5.0,7.0,4.0,7.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.0


In [9]:
#Keep only the necessary rows, rename columns appropriately and cast the float values column to integer
df_enrol = df_enrol[['Row Labels','Grand Total']]
df_enrol = df_enrol.rename(columns={"Row Labels": "school", "Grand Total": "enrolments"})
df_enrol.enrolments = df_enrol.enrolments.astype(int)
df_enrol.head()

,school,enrolments
0,Abbotsford Primary School,144
1,Aberfeldie Primary School,398
2,Ainslie Parklands Primary School,147
3,Aireys Inlet Primary School,90
4,Airly Primary School,33


In [10]:
#Check for any null values
df_enrol.isnull().sum()

school        0
enrolments    0
dtype: int64

In [11]:
# The code was removed by Watson Studio for sharing.

,Education_Sector,Entity_Type,School_No,School_Name,School_Type,School_Status,Address_Line_1,Address_Line_2,Address_Town,Address_State,Address_Postcode,Postal_Address_Line_1,Postal_Address_Line_2,Postal_Town,Postal_State,Postal_Postcode,Full_Phone_No,LGA_ID,LGA_Name,X,Y
0,Government,1,1,Alberton Primary School,Primary,O,21 Thomson Street,NaN,Alberton,VIC,3971,21 Thomson Street,NaN,ALBERTON,VIC,3971,03 5183 2412,681,Wellington (S),146.666601,-38.617713
1,Government,1,3,Allansford and District Primary School,Primary,O,Frank Street,NaN,Allansford,VIC,3277,Frank Street,NaN,ALLANSFORD,VIC,3277,03 5565 1382,673,Warrnambool (C),142.590393,-38.386281
2,Government,1,4,Avoca Primary School,Primary,O,118 Barnett Street,NaN,Avoca,VIC,3467,P O Box 12,NaN,AVOCA,VIC,3467,03 5465 3176,599,Pyrenees (S),143.475649,-37.084502
3,Government,1,8,Avenel Primary School,Primary,O,40 Anderson Street,NaN,Avenel,VIC,3664,40 Anderson Street,NaN,AVENEL,VIC,3664,03 5796 2264,643,Strathbogie (S),145.234722,-36.901368
4,Government,1,12,Warrandyte Primary School,Primary,O,5-11 Forbes Street,NaN,Warrandyte,VIC,3113,5-11 Forbes Street,NaN,WARRANDYTE,VIC,3113,03 9844 3537,421,Manningham (C),145.213980,-37.742675


In [12]:
#Keep only school name and postcode data
df_schoolpostcodes = df_schoollocations[['School_Name', 'Postal_Postcode']]
df_schoolpostcodes = df_schoolpostcodes.rename(columns={"School_Name":"school","Postal_Postcode":"postcode"})
df_schoolpostcodes.reset_index(drop=True, inplace=True)
df_schoolpostcodes.head()

,school,postcode
0,Alberton Primary School,3971
1,Allansford and District Primary School,3277
2,Avoca Primary School,3467
3,Avenel Primary School,3664
4,Warrandyte Primary School,3113


In [13]:
#Keep only postcode data in Metropolitan Melbourne
df_schoolpostcodes = df_schoolpostcodes.loc[((df_schoolpostcodes['postcode'] >= 3000) & (df_schoolpostcodes['postcode'] <= 3207))|((df_schoolpostcodes['postcode'] >= 8000) & (df_schoolpostcodes['postcode'] <= 8873))]
df_schoolpostcodes.head()

,school,postcode
4,Warrandyte Primary School,3113
11,Cheltenham Primary School,3192
13,Newport Lakes Primary School,3015
23,Doncaster Primary School,3108
25,Eltham Primary School,3095


In [14]:
#Merge the postcodes and enrolment dataframes along the "school" column
df_schools = pd.merge(df_enrol, df_schoolpostcodes)
df_schools.head()

,school,enrolments,postcode
0,Abbotsford Primary School,144,3067
1,Aberfeldie Primary School,398,3040
2,Ainslie Parklands Primary School,147,3136
3,Aitken Creek Primary School,1100,3064
4,Alamanda K-9 College,2897,3030


In [15]:
#Group the multiple schools in unique postcodes into total enrolments per postcode
df_enroltotals = df_schools.groupby(['postcode']).sum()
df_enroltotals = df_enroltotals.reset_index()
df_enroltotals.head()

,postcode,enrolments
0,3004,1044
1,3006,861
2,3011,501
3,3012,1952
4,3013,2217


In [16]:
#Perform outer join and fill any NaN values as 0
df_melbenrol = pd.merge(df_melbpostcodes,df_enroltotals,how='outer')
df_melbenrol['enrolments'].fillna(0, inplace=True)
df_melbenrol.enrolments = df_melbenrol.enrolments.astype(int)
df_melbenrol.head()

,postcode,suburb,lat,lon,enrolments
0,3000,MELBOURNE,-37.814563,144.970267,0
1,3001,MELBOURNE,-38.365017,144.765920,0
2,3002,EAST MELBOURNE,-37.816640,144.987811,0
3,3003,WEST MELBOURNE,-37.806255,144.941123,0
4,3004,MELBOURNE,-37.837324,144.976335,1044


In [17]:
#Check for any null values in the new dataframe
df_melbenrol.isnull().sum()

postcode      0
suburb        1
lat           1
lon           1
enrolments    0
dtype: int64

In [18]:
#Check what the row is
df_melbenrol[df_melbenrol.isna().any(axis=1)]

,postcode,suburb,lat,lon,enrolments
230,8008,NaN,NaN,NaN,58


In [19]:
#Dump the row because it's only 58 enrolments (negligible)
df_melbenrol = df_melbenrol.dropna()

### SOCIO-ECONOMIC DATA WRANGLING

In [20]:
# The code was removed by Watson Studio for sharing.

,POA,Postal Area Code,INDEX_TYPE,Index type,MEASURE,Measure,TIME,Time,Value,Flag Codes,Flags
0,800,800,IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,2011,2011,1072.0,NaN,NaN
1,810,810,IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,2011,2011,1037.0,NaN,NaN
2,812,812,IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,2011,2011,1019.0,NaN,NaN
3,820,820,IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,2011,2011,1076.0,NaN,NaN
4,822,822,IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,2011,2011,702.0,NaN,NaN


In [22]:
#Check for duplicates of postcodes
df_data_2['Postal Area Code'].value_counts()

2047    40
2196    40
2452    40
2388    40
4375    40
2324    40
4311    40
6358    40
2260    40
4183    40
3797    40
6230    40
2132    40
4119    40
6166    40
2068    40
6102    40
4053    40
4503    40
2516    40
4567    40
2580    40
3284    40
5271    40
3220    40
7254    40
3156    40
7190    40
3028    40
5015    40
853     40
2900    40
4887    40
2836    40
4823    40
4695    40
2644    40
3925    40
3733    40
3540    40
4374    40
4630    40
2581    40
4566    40
2517    40
4502    40
2453    40
6423    40
2325    40
3669    40
6359    40
4310    40
2261    40
2197    40
2133    40
6167    40
4118    40
2645    40
4694    40
6743    40
2773    40
5654    40
3477    40
5462    40
3413    40
3285    40
5270    40
3221    40
7255    40
5142    40
3093    40
3029    40
5014    40
852     40
4886    40
4822    40
3412    40
3732    40
2641    40
3991    40
2198    40
2134    40
2070    40
4117    40
6164    40
4055    40
6100    40
6036    40
3223    40
3927    40
3799    40

In [21]:
#Drop all duplicates, keeping only those with a relevant SES score
df_ses = df_data_2.drop_duplicates(subset = ['Postal Area Code'], keep = 'first')
df_ses.head()

,POA,Postal Area Code,INDEX_TYPE,Index type,MEASURE,Measure,TIME,Time,Value,Flag Codes,Flags
0,800,800,IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,2011,2011,1072.0,NaN,NaN
1,810,810,IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,2011,2011,1037.0,NaN,NaN
2,812,812,IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,2011,2011,1019.0,NaN,NaN
3,820,820,IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,2011,2011,1076.0,NaN,NaN
4,822,822,IRSAD,Index of Relative Socio-economic Advantage and...,SCORE,Score,2011,2011,702.0,NaN,NaN


In [22]:
#Keep only the relevant columns of postcode and SES score, renaming the columns appropriately
df_ses = df_ses[['Postal Area Code','Value']]
df_ses = df_ses.rename(columns = {"Postal Area Code":"postcode","Value":"ses_score"})
df_ses.head(10)

,postcode,ses_score
0,800,1072.0
1,810,1037.0
2,812,1019.0
3,820,1076.0
4,822,702.0
5,828,972.0
6,829,948.0
7,830,996.0
8,832,1076.0
9,835,1054.0


In [23]:
# Merging SES data with main dataframe
df_melbsesenrol = pd.merge(df_melbenrol, df_ses, how='outer')
df_melbsesenrol.head(10)

,postcode,suburb,lat,lon,enrolments,ses_score
0,3000,MELBOURNE,-37.814563,144.970267,0.0,1030.0
1,3001,MELBOURNE,-38.365017,144.765920,0.0,NaN
2,3002,EAST MELBOURNE,-37.816640,144.987811,0.0,1126.0
3,3003,WEST MELBOURNE,-37.806255,144.941123,0.0,1088.0
4,3004,MELBOURNE,-37.837324,144.976335,1044.0,1116.0
5,3005,WORLD TRADE CENTRE,-37.822262,144.954856,0.0,1104.0
6,3006,SOUTHBANK,-37.823258,144.965926,861.0,1110.0
7,3008,DOCKLANDS,-37.814719,144.948039,0.0,1115.0
8,3010,UNIVERSITY OF MELBOURNE,-37.796152,144.961351,0.0,NaN
9,3011,"FOOTSCRAY, SEDDON, SEDDON WEST",-37.801199,144.887090,501.0,980.0


In [24]:
#Filter for only the postcodes of Metropolitan Melbourne
df_melbsesenrol = df_melbsesenrol.loc[((df_melbsesenrol['postcode'] >= 3000) & (df_melbsesenrol['postcode'] <= 3207))|((df_melbsesenrol['postcode'] >= 8000) & (df_melbsesenrol['postcode'] <= 8873))]
df_melbsesenrol.head()

,postcode,suburb,lat,lon,enrolments,ses_score
0,3000,MELBOURNE,-37.814563,144.970267,0.0,1030.0
1,3001,MELBOURNE,-38.365017,144.765920,0.0,NaN
2,3002,EAST MELBOURNE,-37.816640,144.987811,0.0,1126.0
3,3003,WEST MELBOURNE,-37.806255,144.941123,0.0,1088.0
4,3004,MELBOURNE,-37.837324,144.976335,1044.0,1116.0


In [25]:
#Display any NaN rows
df_melbsesenrol[df_melbsesenrol.isnull().any(axis=1)]

,postcode,suburb,lat,lon,enrolments,ses_score
1,3001,MELBOURNE,-38.365017,144.765920,0.0,NaN
8,3010,UNIVERSITY OF MELBOURNE,-37.796152,144.961351,0.0,NaN
45,3050,ROYAL MELBOURNE HOSPITAL,-37.798631,144.955627,0.0,NaN
57,3062,SOMERTON,-37.642563,144.944259,1924.0,NaN
78,3086,LA TROBE UNIVERSITY,-37.721328,145.047012,0.0,NaN
146,3164,DANDENONG SOUTH,-38.022430,145.237380,0.0,NaN
188,8001,MELBOURNE,-38.365017,144.765920,0.0,NaN
189,8002,EAST MELBOURNE,-38.105449,145.147855,0.0,NaN
190,8004,ST KILDA ROAD,-37.836219,144.975549,0.0,NaN
191,8005,WORLD TRADE CENTRE,-37.822262,144.954856,0.0,NaN


In [26]:
#Find SEIFA scores for the postcodes near 3062
border = ['CRAIGIEBURN', 'ROXBURGH PARK', 'MEADOW HEIGHTS', 'COOLAROO', 'CAMPBELLFIELD', 'LALOR', 'EPPING']
SEIFA = []
suburbs_found = []

for b in border:
    SEIFA.append(df_melbsesenrol[df_melbsesenrol['suburb'].str.contains(b)].iloc[0,5])
    suburbs_found.append(df_melbsesenrol[df_melbsesenrol['suburb'].str.contains(b)].iloc[0,1])

In [27]:
#Display the array of SEIFA scores
SEIFA

[965.0, 965.0, 834.0, 834.0, 827.0, 885.0, 974.0]

In [28]:
#Display all of the suburbs collected from the exercise
suburbs_found

['CRAIGIEBURN, DONNYBROOK, KALKALLO, MICKLEHAM, ROXBURGH PARK',
 'CRAIGIEBURN, DONNYBROOK, KALKALLO, MICKLEHAM, ROXBURGH PARK',
 'COOLAROO, MEADOW HEIGHTS',
 'COOLAROO, MEADOW HEIGHTS',
 'CAMPBELLFIELD',
 'LALOR, LALOR PLAZA',
 'EPPING']

In [29]:
#Calculate the average SEIFA score only for unique postcodes and then update postcode 3062 with that value
average = (965+834+827+885+974)/5
df_melbsesenrol.loc[df_melbsesenrol.postcode == 3062, 'ses_score'] = average

#Then check for NaNs to see that it is resolved
df_melbsesenrol[df_melbsesenrol.isnull().any(axis=1)]

,postcode,suburb,lat,lon,enrolments,ses_score
1,3001,MELBOURNE,-38.365017,144.765920,0.0,NaN
8,3010,UNIVERSITY OF MELBOURNE,-37.796152,144.961351,0.0,NaN
45,3050,ROYAL MELBOURNE HOSPITAL,-37.798631,144.955627,0.0,NaN
78,3086,LA TROBE UNIVERSITY,-37.721328,145.047012,0.0,NaN
146,3164,DANDENONG SOUTH,-38.022430,145.237380,0.0,NaN
188,8001,MELBOURNE,-38.365017,144.765920,0.0,NaN
189,8002,EAST MELBOURNE,-38.105449,145.147855,0.0,NaN
190,8004,ST KILDA ROAD,-37.836219,144.975549,0.0,NaN
191,8005,WORLD TRADE CENTRE,-37.822262,144.954856,0.0,NaN
192,8006,ABECKETT STREET,-37.809696,144.959314,0.0,NaN


In [30]:
#Drop the rest of the NaN values
df_melbsesenrol = df_melbsesenrol.dropna()
df_melbsesenrol.head()

,postcode,suburb,lat,lon,enrolments,ses_score
0,3000,MELBOURNE,-37.814563,144.970267,0.0,1030.0
2,3002,EAST MELBOURNE,-37.816640,144.987811,0.0,1126.0
3,3003,WEST MELBOURNE,-37.806255,144.941123,0.0,1088.0
4,3004,MELBOURNE,-37.837324,144.976335,1044.0,1116.0
5,3005,WORLD TRADE CENTRE,-37.822262,144.954856,0.0,1104.0


### FOURSQUARE TUITION VENUE DATA WRANGLING

In [31]:
# The code was removed by Watson Studio for sharing.

In [49]:
search_query = 'VCE'
def getNearbyVenues(names, latitudes, longitudes, radius=2000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):    
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}'.format(
        CLIENT_ID, 
        CLIENT_SECRET, 
        lat, 
        lng,
        ACCESS_TOKEN, 
        VERSION, 
        search_query, 
        radius, 
        LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]["venues"]
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['name'],
            v['location']['lat'], 
            v['location']['lng'],  
            ) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood_Latitude', 
                  'Neighbourhood_Longitude', 
                  'Venue',
                  'Venue_Latitude',
                  'Venue_Longitude']
    
    return(nearby_venues)

In [50]:
melb_venues = getNearbyVenues(names=df_melbsesenrol['suburb'],
                                   latitudes=df_melbsesenrol['lat'],
                                   longitudes=df_melbsesenrol['lon']
                                   )
melb_venues.head()

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude
0,MELBOURNE,-37.814563,144.970267,VCE Centre,-37.815353,144.963237
1,MELBOURNE,-37.814563,144.970267,VCE Tuition Centre,-37.815460,144.963280
2,MELBOURNE,-37.814563,144.970267,vce tuition centre,-37.813533,144.960043
3,MELBOURNE,-37.814563,144.970267,TSSM - Creating VCE Success,-37.819600,144.957599
4,MELBOURNE,-37.814563,144.970267,Gedu 环球教育,-37.814615,144.960591


In [51]:
# The code was removed by Watson Studio for sharing.

{'file_name': 'melb_venues_vce.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'applieddatasciencecapstone-donotdelete-pr-iro9kxis5xquju',
 'asset_id': 'debf4748-013a-4f62-a334-564735256dab'}

In [36]:
# The code was removed by Watson Studio for sharing.

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude
0,MELBOURNE,-37.814563,144.970267,VCE Tuition Centre,-37.815460,144.963280
1,MELBOURNE,-37.814563,144.970267,Alchemy Tuition,-37.812680,144.962730
2,MELBOURNE,-37.814563,144.970267,RL TUITION,-37.812298,144.961493
3,MELBOURNE,-37.814563,144.970267,vce tuition centre,-37.813533,144.960043
4,EAST MELBOURNE,-37.816640,144.987811,VCE Tuition Centre,-37.815460,144.963280


In [37]:
# The code was removed by Watson Studio for sharing.

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude
0,MELBOURNE,-37.814563,144.970267,ESL Tutoring,-37.813149,144.962942
1,MELBOURNE,-37.814563,144.970267,Captivate Tutoring,-37.818294,144.956676
2,EAST MELBOURNE,-37.816640,144.987811,ESL Tutoring,-37.813149,144.962942
3,WEST MELBOURNE,-37.806255,144.941123,ESL Tutoring,-37.813149,144.962942
4,WEST MELBOURNE,-37.806255,144.941123,Captivate Tutoring,-37.818294,144.956676


In [53]:
# The code was removed by Watson Studio for sharing.

,Neighbourhood,Neighbourhood_Latitude,Neighbourhood_Longitude,Venue,Venue_Latitude,Venue_Longitude
0,MELBOURNE,-37.814563,144.970267,VCE Centre,-37.815353,144.963237
1,MELBOURNE,-37.814563,144.970267,VCE Tuition Centre,-37.815460,144.963280
2,MELBOURNE,-37.814563,144.970267,vce tuition centre,-37.813533,144.960043
3,MELBOURNE,-37.814563,144.970267,TSSM - Creating VCE Success,-37.819600,144.957599
4,MELBOURNE,-37.814563,144.970267,Gedu 环球教育,-37.814615,144.960591


In [54]:
melb_allvenues = pd.concat([df_venues_tuition,df_venues_tutoring,df_venues_vce])
melb_allvenues.shape

(145, 6)

In [55]:
melb_allvenues = melb_allvenues.drop_duplicates(subset=['Venue','Venue_Latitude','Venue_Longitude'])
melb_allvenues.shape

(33, 6)

In [56]:
def haversine_np(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = np.sin(dlat / 2.0) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2.0) ** 2

    c = 2 * np.arcsin(np.sqrt(a))
    km = 6367 * c
    return km

In [58]:
venue_numbers = []
for postcode in df_melbsesenrol.itertuples():
    venues = 0
    for venue in melb_allvenues.itertuples():
        if (haversine_np(venue.Venue_Longitude,venue.Venue_Latitude,postcode.lon,postcode.lat) < 5):
            venues = venues+1
    venue_numbers.append(venues)

In [59]:
df_melbdata = df_melbsesenrol
df_melbdata['tuition_venues'] = venue_numbers
df_melbdata.head(50)

,postcode,suburb,lat,lon,enrolments,ses_score,tuition_venues
0,3000,MELBOURNE,-37.814563,144.970267,0.0,1030.0,11
2,3002,EAST MELBOURNE,-37.816640,144.987811,0.0,1126.0,11
3,3003,WEST MELBOURNE,-37.806255,144.941123,0.0,1088.0,10
4,3004,MELBOURNE,-37.837324,144.976335,1044.0,1116.0,11
5,3005,WORLD TRADE CENTRE,-37.822262,144.954856,0.0,1104.0,10
6,3006,SOUTHBANK,-37.823258,144.965926,861.0,1110.0,11
7,3008,DOCKLANDS,-37.814719,144.948039,0.0,1115.0,10
9,3011,"FOOTSCRAY, SEDDON, SEDDON WEST",-37.801199,144.887090,501.0,980.0,2
10,3012,"BROOKLYN, KINGSVILLE, KINGSVILLE WEST, MAIDSTO...",-37.800197,144.867860,1952.0,973.0,3
11,3013,"YARRAVILLE, YARRAVILLE WEST",-37.817099,144.886678,2217.0,1054.0,2
